# Grafote: ¿Cuáles son mis notas menos confiables?

Este grafo se encarga de analizar las notas asignadas por parte de las materias hacia estudiantes con el objetivo de identificar las materias injustas, notas confiables, mejores estudiantes, y más. Para esto se utiliza el algoritmo _REV2_.

_REV2_ tiene como objetivo encontrar usuarios fraudulentos en plataformas que tengan ratings hacia ciertos productos.

Bajo el contexto de FIUBA, podríamos pensar a las materias como "usuarios", a los estudiantes como "productos", y las notas de final como "calificaciones" con el objetivo de aplicar REV2 _con algunos supuestos_:

- Un estudiante mantiene un cierto desempeño desde su inicio a fin de carrera.
- La forma de evaluar de cada materia se mantiene consistente más allá de cátedras o profesores.

No esperamos encontrar _materias fraudulentas_, pero de cualquier forma podemos analizar los valores F, G y R para materias, usuarios y notas que devuelve el algoritmo.

## ¿Cómo es el grafo?

El grafo va a ser bipartito

- Nodos: Materias por un lado, estudiantes por el otro
- Aristas: Un estudiante aprobó una materia, la arista representa la nota y está direccionada de la materia hacia el estudiante (materia valoró a este estudiante con esta nota)
- Peso: Nota de final

A diferencia de los otros grafos, este utiliza la información de _todas_ las carreras en vez de limitarse a solo una.

## Preprocesamiento

In [ ]:
import networkx as nx
import pandas as pd
import numpy as np
import scipy as sp
from tqdm.notebook import tqdm

import utils

Obtenemos todos los planes de carrera de FIUBA, para tener los nombres de materias al analizar los resultados

In [ ]:
df_planes = []
for plan in utils.PLANES:
    df_plan = pd.read_json(utils.plan_estudios(plan))
    df_plan['Carrera'] = plan
    df_planes.append(df_plan)
df_planes = pd.concat(df_planes)
df_planes = df_planes.rename(columns={'materia': 'materia_nombre', 'id': 'materia_id'})
df_planes.sample(3)

Y el grafo con las notas en sí

In [ ]:
df = pd.read_pickle('fiuba-map-data-all.pickle')
display(df.sample(3))
df.shape

In [ ]:
# join con los planes, para tenerlo
df = pd.merge(df, df_planes, on=['materia_id', 'Carrera'])

# filtros de columnas relevantes
df = df[['Padron', 'materia_id', 'materia_nota', 'materia_nombre', 'Carrera']]
df.head()

In [ ]:
# Se toman solo cursadas aprobadas
df_notas = df[df['materia_nota'] >= 4].copy()

# Se ignora el CBC
df_notas = df_notas[~df_notas['materia_id'].str.contains('CBC')]

df_notas['nota_mediana'] = df_notas.groupby(['Padron', 'Carrera'])['materia_nota'].transform('median')
df_notas['nota_promedio'] = df_notas.groupby(['Padron', 'Carrera'])['materia_nota'].transform('mean')
df_notas['nota_diferentes'] = df_notas.groupby(['Padron', 'Carrera'])['materia_nota'].transform('nunique')

# Para reducir ruido, se eliminan mapas donde las notas son casi "todas iguales"
df_notas = df_notas[df_notas['nota_diferentes'] >= 4]
df_notas = df_notas[df_notas['nota_mediana'] > 4.5]

df_notas.shape

In [ ]:
# Normalizar notas de [-1 a 1]
df_notas['materia_nota_norm'] = (df_notas['materia_nota'] - 7) / 3
df_notas['materia_nota_norm'].value_counts()

In [ ]:
# Se agregan columnas de cantidades, para luego filtrar más adelante
df_notas['cant_materias'] = df_notas.groupby(['Padron'])['materia_id'].transform('count')
df_notas['cant_padrones'] = df_notas.groupby(['materia_id'])['Padron'].transform('count')

## Algoritmo REV2

Se plantea el algoritmo genérico de REV2 para luego aplicarlo al grafo armado.

In [ ]:
def rev2(
    df,
    col_in,
    col_out,
    col_rating,
    gamma_1=0.5,
    gamma_2=0.5,
    n_iter=20,
):
    """Algoritmo REV2. Dado un dataframe con entradas, salidas y ratings
    denotados por los nombres de columna, devuelve series para aplicar los
    valores de F, G y R.
    """
    assert df[col_rating].min() == -1 and df[col_rating].max() == 1, "Rating no fue normalizado a [-1, 1]"

    ratings = nx.from_pandas_edgelist(
        df,
        create_using=nx.DiGraph,
        source=col_in,
        target=col_out,
        edge_attr=[col_rating]
    )
    
    F = {}
    G = {}

    for val_in, val_out in tqdm(ratings.edges(), 'Inicializando estructuras'):
        F[val_in] = 1
        G[val_out] = 1
        ratings[val_in][val_out]['R'] = 1

    for i in tqdm(range(n_iter), 'Realizando iteraciones F/G/R'):
        for val_out in G:
            s = 0
            n = 0
            for val_in in ratings.predecessors(val_out):
                s += ratings[val_in][val_out]['R'] * ratings[val_in][val_out][col_rating]
                n += 1
            G[val_out] = s / n
            assert -1 <= G[val_out] <= 1

        for val_in, val_out in ratings.edges():
            R_new = (gamma_1 * F[val_in] + gamma_2 * (1 - (abs(ratings[val_in][val_out][col_rating] - G[val_out]) / 2))) / (gamma_1 + gamma_2)
            ratings[val_in][val_out]['R'] = R_new
            assert 0 <= R_new <= 1

        for val_in in F:
            s = 0
            for val_out in ratings[val_in]:
                s += ratings[val_in][val_out]['R']
            F[val_in] = s / len(ratings[val_in])
            assert 0 <= F[val_in] <= 1

    series_F = df[col_in].apply(lambda x: F[x])
    series_G = df[col_out].apply(lambda x: G[x])
    series_R = df[[col_in, col_out]].apply(lambda x: ratings[x[col_in]][x[col_out]]['R'], axis=1)

    return series_F, series_G, series_R, ratings

In [ ]:
series_F, series_G, series_R, G = rev2(
    df_notas,
    'materia_id',
    'Padron',
    'materia_nota_norm',
)

In [ ]:
df_notas['F'] = series_F
df_notas['G'] = series_G
df_notas['R'] = series_R

In [ ]:
# Veamos (un sample de) el grafo sobre el cual se corrió REV2
import matplotlib.pyplot as plt
import utils
from config import PADRON
import random

# Subgrafo => 3 materias troncales y 3 padrones al azar
materias = ["62.01", "61.03", "61.08"]
random_edges = random.choices(list(filter(lambda e: any([m in e for m in materias]), G.edges)), k=3)
random_padrones = [p for m, p in random_edges]

nodes = [*random_padrones, *materias]
subgraph = nx.subgraph(G, nodes)

plt.figure(figsize=(30,10))
plt.title(f"Sample of a {G}")
nx.draw_networkx(subgraph,
                 pos=nx.bipartite_layout(subgraph, nx.bipartite.sets(subgraph)[0]))

## Análisis de resultados

Para el análisis de materias, se consideran solo aquellas que tengan un mínimo de estudiantes aprobados. Lo mismo para el análisis de estudiantes

In [ ]:
MATERIAS_APROBADAS = 7
ESTUDIANTES_APROBADOS = 15

In [ ]:
# Para ver los nombres completos de los nombres de materias
pd.set_option('display.max_colwidth', None)

### Fairness

Bajo este contexto, una materia _injusta_ puede dar nota baja a "buenos estudiantes" y/o nota alta a "malos estudiantes".

In [ ]:
(
    df_notas[(df_notas['cant_padrones'] >= ESTUDIANTES_APROBADOS)]
        [['materia_id', 'materia_nombre', 'F']]
        .drop_duplicates()
        .sort_values('F', ascending=True)
        .head(15)
)

Las materias _justas_ suelen asignar notas conservadoras que tienden de 6 a 8.

In [ ]:
(
    df_notas[(df_notas['cant_padrones'] >= ESTUDIANTES_APROBADOS)]
        [['materia_id', 'materia_nombre', 'F']]
        .drop_duplicates()
        .sort_values('F', ascending=False)
        .head(15)
)

### Goodness

Nota que una materia "confiable" le podría dar a tal estudiante. No es lo mismo que el promedio pero existe una correlación.

In [ ]:
df_notas['G_norm'] = df_notas['G'] * 3 + 7
(
    df_notas[(df_notas['cant_materias'] >= MATERIAS_APROBADAS)]
        [['Padron', 'Carrera', 'G_norm', 'nota_promedio']]
        .drop_duplicates()
        .sort_values('G_norm', ascending=True)
        .head(15)
)

In [ ]:
(
    df_notas[(df_notas['cant_materias'] >= MATERIAS_APROBADAS)]
        [['Padron', 'Carrera', 'G_norm', 'nota_promedio']]
        .drop_duplicates()
        .sort_values('G_norm', ascending=False)
        .head(15)
)

### Reliability

Una nota poco confiable puede estar en dos extremos: una nota alta a un estudiante con un desempeño general bajo, o una nota baja a un estudiante con un buen desempeño

In [ ]:
(
    df_notas[
        (df_notas['cant_padrones'] >= ESTUDIANTES_APROBADOS)
        & (df_notas['cant_materias'] >= MATERIAS_APROBADAS)
    ][['Padron', 'Carrera', 'materia_id', 'materia_nombre', 'materia_nota', 'R', 'G_norm']]
        .drop_duplicates()
        .sort_values('R', ascending=True)
        .head(15)
)

Una nota confiable es la más esperable para asignarle al estudiante según su valor de G.

In [ ]:
(
    df_notas[
        (df_notas['cant_padrones'] >= ESTUDIANTES_APROBADOS)
        & (df_notas['cant_materias'] >= MATERIAS_APROBADAS)
    ][['Padron', 'Carrera', 'materia_id', 'materia_nombre', 'materia_nota', 'R', 'G_norm']]
        .drop_duplicates()
        .sort_values('R', ascending=False)
        .head(15)
)

### Reliability de un padrón particular

Considerando un único padrón podemos ver sus notas "menos confiable" considerando el cuantil 0.1 de R

In [ ]:
(
    df_notas[
        (df_notas['Padron'] == utils.PADRON)
        & (df_notas['R'] <= df_notas['R'].quantile(0.10))
    ][['Padron', 'Carrera', 'materia_id', 'materia_nombre', 'materia_nota', 'R', 'G_norm']]
        .drop_duplicates()
        .sort_values('R', ascending=True)
        .head(5)
)